In [9]:
import pandas as pd
import redis
from redis.commands.search.field import TextField, NumericField
from redis.commands.search.index_definition import IndexDefinition, IndexType

In [10]:
df = pd.read_csv("channels.csv")
df = df.fillna("")

In [11]:
df.head()

,Sr. No.,Channel Name,HD/SD,Genre,EPG No.,FTA/Pay,MRP,MRP + Tax
0,1,DD National,SD,Hindi Entertainment,114,FTA,FTA,FTA
1,2,Star Plus HD,HD,Hindi Entertainment,115,Pay,19,22.42
2,3,Star Plus,SD,Hindi Entertainment,117,Pay,19,22.42
3,4,Star Bharat HD,HD,Hindi Entertainment,121,Pay,19,22.42
4,5,Star Bharat,SD,Hindi Entertainment,122,Pay,10,11.8


In [12]:
r = redis.Redis(host="localhost", port=6379, decode_responses=True)


In [13]:
try:
    r.ft("channels").create_index(
        fields=[
            TextField("search_text"),
            TextField("Channel_Name"),
            TextField("Genre"),
            TextField("HD_SD"),
            NumericField("EPG_No"),
            TextField("FTA_Pay"),
            TextField("MRP"),
            TextField("MRP_Tax")
        ],
        definition=IndexDefinition(prefix=["channel:"], index_type=IndexType.HASH)
    )
    print("Index created successfully.")
except Exception as e:
    print("Index creation failed or already exists:", e)

Index created successfully.


In [14]:
for idx, row in df.iterrows():
    channel_id = f"channel:{idx}"
    search_text = f"{row['Channel Name']} {row['Genre']}"
    
    r.hset(channel_id, mapping={
        "search_text": str(search_text),
        "Channel_Name": str(row["Channel Name"]),
        "Genre": str(row["Genre"]),
        "HD_SD": str(row["HD/SD"]),
        "EPG_No": int(row["EPG No."]) if str(row["EPG No."]).isdigit() else -1,
        "FTA_Pay": str(row["FTA/Pay"]),
        "MRP": str(row["MRP"]),
        "MRP_Tax": str(row["MRP + Tax"])
    })
